In [ ]:
addprocs(5)

In [ ]:
using WordDistributions
using Utils
using WordEmbeddings                                                                                                                                            
using Training                                                                                                                                                  
using Query
using SoftmaxClassifier
using WordStreams

In [ ]:
model_dir = "../eval/models/ss"                                                                                                                                         
test_file = "../test/data/text8_tiny"                                                                                                                
const ndims = 25                                                                                                                                                

embed = WordSenseEmbedding(ndims, random_inited, huffman_tree, subsampling = 0.0, min_count=1)

In [ ]:
embed.distribution = word_distribution(test_file)
embed.vocabulary = collect(keys(embed.distribution))

initialize_embedding(embed, embed.init_type)        # initialize by the specified method
Training.initialize_network(embed, embed.network_type)

# determine the position in the tree for every word
for (w, code) in Trees.leaves_of(embed.classification_tree)
    embed.codebook[w] = code
end
print("done")

In [ ]:
words = split("the all say the king is dead long live the king")
word = words[end÷2]
context = [words[1:end÷2-1];words[end÷2+1:end]] 

In [ ]:
windows= sliding_window(words_of(test_file, embed.distribution))

In [ ]:
first(windows)[end÷2]

In [ ]:
ch = RemoteChannel()

In [ ]:
@time remotecall(default_worker_pool(),embed,word,context, ch) do i_embed, i_word, i_context, i_ch
    sense_id = Training.WSD(i_embed, i_word, i_context)
    put!(i_ch, sense_id)
end

In [ ]:
take!(ch)

In [ ]:
addprocs(5)

In [ ]:

function r(ch) 
    wkpool = default_worker_pool()
    remotecall(wkpool, ch) do channel
        for ii in 1:4
            put!(channel,((myid(),ii)))
            sleep(1)
        end
    end
    wkpool
end


In [ ]:
c1 = RemoteChannel()


In [2]:
take!(c1)

LoadError: LoadError: UndefVarError: c1 not defined
while loading In[2], in expression starting on line 1

In [3]:
using SoftmaxClassifier

In [4]:

@fastmath function predict1{F<:AbstractFloat}(c::LinearClassifier{2},
											 x::AbstractVector{F})
    t1=zero(F)
    t2=zero(F)
    @inbounds for ii in 1: size(c.weights,1)
        @inbounds t1+=c.weights[ii,1]*x[ii]
        @inbounds t2+=c.weights[ii,2]*x[ii]
    end
    return SoftmaxClassifier.softmax2(t1,t2)
end

predict1 (generic function with 1 method)

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31


In [6]:
cc=LinearClassifier(2,50);
@time map(ii->predict1(cc, rand(50)), 1:100);

  0.040944 seconds (17.15 k allocations: 783.795 KB)


In [7]:
@fastmath function predict2{F<:AbstractFloat}(c::LinearClassifier{2},
    										 x::AbstractVector{F})
    t1= Threads.Atomic{F}()
    t2= Threads.Atomic{F}()
    t1[]=zero(F)
    t2[]=zero(F)
    Threads.@threads for ii in 1: size(c.weights,1)
        @inbounds c1 = c.weights[ii,1]
        @inbounds c2 = c.weights[ii,2]
        Threads.atomic_add!(t1,c1*x[ii])
        Threads.atomic_add!(t2,c2*x[ii])
    end
    return SoftmaxClassifier.softmax2(t1[],t2[])
end

predict2 (generic function with 1 method)

In [10]:
cc=LinearClassifier(2,50);
@time map(ii->predict2(cc, rand(50)), 1:100);

  1.678831 seconds (17.55 k allocations: 794.706 KB)


In [ ]:
Threads.nthreads()

In [ ]:

Threads.@threads for i=1:100; rand(i) ; end

In [ ]:
Threads.nthreads()